In [1]:
import numpy as np
import pandas as pd

from scipy.stats import wilcoxon

In [2]:
estimator = 'dt'
method = 'c'
postfix = '_mid'

token = f'{estimator}{method}{postfix}'

data = pd.concat([
    pd.read_csv(f'effect_presence_{token}.csv'),
    pd.read_csv(f'proposed_a{token}.csv'),
    pd.read_csv(f'proposed_f{token}.csv')
])

In [6]:
score = 'auc' if 'auc' in data.columns else 'r2'

In [7]:
def labeling(row):
    params = eval(row['params'])
    if row['estimator'].startswith('Operator'):
        if params['operator'] == '<':
            return 'orig_l'
        else:
            return 'orig_leq'
    if row['estimator'].startswith('Averaged'):
        mode = params.get('mode', 'def')
        return f'avg_{mode}'
    if row['estimator'].startswith('Flipping'):
        mode = params.get('mode', 'def')
        return f'flip_{mode}'

def regularization(row):
    params = eval(row['params'])
    return params.get('min_impurity_decrease')

In [8]:
data['label'] = data.apply(labeling, axis=1)
data['min_samples_leaf'] = data.apply(regularization, axis=1)

In [9]:
perf = data.groupby(['name', 'label', 'min_samples_leaf'], dropna=False).agg({score: 'mean'}).reset_index(drop=False)

In [10]:
filter = perf[perf['label'] == 'orig_l'].groupby(['name']).apply(lambda row: row.sort_values(score).iloc[-1][['name', 'min_samples_leaf']]).reset_index(drop=True)

In [11]:
filter

,name,min_samples_leaf
0,abalone9_18,0.20
1,appendicitis,0.20
2,bupa,0.05
3,cleveland-0_vs_4,0.05
4,ecoli1,0.10
5,glass0,0.10
6,haberman,0.20
7,hepatitis,0.05
8,lymphography,NaN
9,mammographic,0.05


In [12]:
data = pd.merge(data, filter, on=['name', 'min_samples_leaf'])

In [13]:
#data = data[data['min_samples_leaf'] == 0.1]

In [14]:
def grouping(pdf):
    pdf = pdf.sort_values(['min_samples_leaf', 'fold'])
    return pdf[score].values.tolist()

In [15]:
grouped = data.groupby(['name', 'label']).apply(grouping).reset_index(drop=False).rename(columns={0: 'scores'})

In [16]:
def evaluate(pdf):
    pivot = 'orig_leq'

    pivot_row = pdf[pdf['label'] == pivot].iloc[0]

    results = {}
    results[f'{score}_orig_leq'] = np.median(pivot_row['scores'])

    for idx, row in pdf.iterrows():
        if row['label'] != pivot:
            results[f'{score}_{row["label"]}'] = np.median(row['scores'])
            if not row['label'].endswith('orig_l'):
                results[f'p_{row["label"]}_l'] = wilcoxon(pivot_row['scores'], row['scores'], zero_method='zsplit'
                                                        , alternative='less'
                                                        ).pvalue
            else:
                results[f'p_{row["label"]}_t'] = wilcoxon(pivot_row['scores'], row['scores'], zero_method='zsplit'
                                                        #, alternative='less'
                                                        ).pvalue

    return pd.Series(results)

In [17]:
grouped.groupby('name').apply(evaluate)

,auc_orig_leq,auc_avg_def,p_avg_def_l,auc_avg_random,p_avg_random_l,auc_flip_def,p_flip_def_l,auc_orig_l,p_orig_l_t
name,,,,,,,,,
abalone9_18,0.764441,0.764441,0.500000,0.764441,0.500000,0.764441,1.410948e-03,0.764441,1.000000
appendicitis,0.772059,0.772059,0.500000,0.772059,0.500000,0.772059,5.000000e-01,0.772059,1.000000
bupa,0.691379,0.692672,0.574377,0.692241,0.679962,0.693319,1.888333e-01,0.690948,0.370077
cleveland-0_vs_4,0.939394,0.939394,0.148217,0.939394,0.148217,0.939394,8.377655e-02,0.941051,0.259088
ecoli1,0.957400,0.957400,0.500000,0.957400,0.500000,0.958960,1.118837e-08,0.957400,1.000000
glass0,0.843596,0.843596,0.500000,0.843596,0.500000,0.845861,3.311841e-04,0.843596,1.000000
haberman,0.666319,0.666319,0.000199,0.666319,0.000199,0.666319,1.112115e-04,0.666667,0.000222
hepatitis,0.760952,0.760952,0.567823,0.760952,0.568308,0.763333,2.993802e-01,0.761905,0.995362
lymphography,1.000000,1.000000,0.298129,1.000000,0.298129,1.000000,9.762029e-02,1.000000,0.596259
